In [1]:
# Install required packages
!pip install transformers flask flask-cors pyngrok --quiet
!pip install sentence-transformers scikit-learn --quiet
!pip install sacremoses --quiet

# Load Models

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load Med42 - Llama3 based medical model (better than BioGPT)
tokenizer = AutoTokenizer.from_pretrained("m42-health/Llama3-Med42-8B")
model = AutoModelForCausalLM.from_pretrained(
    "m42-health/Llama3-Med42-8B",
    torch_dtype="auto",
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Med42 model already uses device_map="auto", so no need to move manually
print(f"🔧 Using device: {device}")
print(f"🤖 Med42 model loaded with automatic device mapping")

🔧 Using device: cuda
🤖 Med42 model loaded with automatic device mapping


In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

biobert = SentenceTransformer('pritamdeka/S-BioBert-snli-multinli-stsb', device=device)

# Document Embeddings & Retriver

In [5]:
docs = []
doc_embeddings = []

In [6]:
def add_doc(text):
    # Embed the incoming doc using BioBERT
    embedding = biobert.encode([text], convert_to_tensor=True)[0].cpu().numpy()

    # Store in memory
    docs.append(text)
    doc_embeddings.append(embedding)

    print(f"✅ Added document. Total docs: {len(docs)}")

In [7]:
def find_most_similar_doc(query, top_k=1):
    if not doc_embeddings:
        return "❌ No documents in memory."

    # Embed the user query
    query_embedding = biobert.encode([query], convert_to_tensor=True)[0].cpu().numpy()

    # Compute cosine similarities
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # Get the top-k most similar docs
    top_indices = similarities.argsort()[-top_k:][::-1]

    results = []
    for idx in top_indices:
        results.append({
            "doc": docs[idx],
            "similarity": float(similarities[idx])
        })

    return results

# API

In [8]:
!pip install flask flask-cors

In [9]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/embed-doc', methods=['POST'])
def embed_doc():
    data = request.get_json()
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "Missing 'text' field."}), 400

    add_doc(text)
    return jsonify({"message": "✅ Document embedded.", "total_docs": len(docs)})

@app.route('/query', methods=['POST'])
def query():
    data = request.get_json()
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "Missing 'query' field."}), 400

    results = find_most_similar_doc(query, top_k=1)
    return jsonify(results)

In [10]:
!pip install pyngrok

In [12]:
from pyngrok import ngrok

# Replace with your token from ngrok.com
ngrok.set_auth_token(token)

In [ ]:
# import os
# import signal
# from pyngrok import ngrok

# # Kill ngrok
# ngrok.kill()

# # Kill the Flask thread by killing the whole Colab process (if needed)
# os.kill(os.getpid(), signal.SIGKILL)

# print('Killed')